In [1]:
import cv2
import numpy as np
import pyautogui
import time
pyautogui.FAILSAFE = False

## Initialisation

In [2]:
# range to pass in cv2 inrange function
blue_range = np.array([[116,45,50],[128,255,255]])
yellow_range = np.array([[17,98,80],[61,255,255]])
red_range = np.array([[47,69,41],[180 ,255,255]])
    
# default initial values for the colored centers
b_cen = [240, 320]
y_pos = [240, 320]
r_cen = [240, 320]

cursor = [960, 540]

# Area ranges for contours of different colours to be detected
r_area = [100, 1600]
b_area = [100, 1600]
y_area = [100, 1600]

# defining square kernel to be passed for transformation using erode and dilate
kernel = np.ones((7,7),np.uint8)

# toggle variables for showing the centroid and performing a specific action
perform = False
showCentroid = False

## Helper functions

In [3]:
# finding the centroid using the cv2 moments function
def findCentroid(contourCoord, showCentroid, vid):
    M = cv2.moments(contourCoord)
    if M['m00'] != 0:
        x_coord = int(M['m10']/M['m00'])
        y_coord = int(M['m01']/M['m00'])
        center = (x_coord,y_coord)
        if showCentroid:
            cv2.circle( vid, center, 5, (0,0,255), -1)

        return center
    

# handling of cursor position based on the parameters passed
def cursorHandler(yp, cursor):
    if yp[0] > 110 and yp[0] < 590 and yp[1] > 120 and yp[1] < 390:
        pyautogui.moveTo(cursor[0],cursor[1])
    elif yp[0] < 110 and yp[1] > 120 and yp[1] < 390:
        pyautogui.moveTo( 8 , cursor[1])
    elif yp[0] > 590 and yp[1] > 120 and yp[1] < 390:
        pyautogui.moveTo(1912, cursor[1])
    elif yp[0] > 110 and yp[0] < 590 and yp[1] < 120:
        pyautogui.moveTo(cursor[0] , 8)
    elif yp[0] > 110 and yp[0] < 590 and yp[1] > 390:
        pyautogui.moveTo(cursor[0] , 1072)
    elif yp[0] < 110 and yp[1] < 120:
        pyautogui.moveTo(8, 8)
    elif yp[0] < 110 and yp[1] > 390:
        pyautogui.moveTo(8, 1072)
    elif yp[0] > 590 and yp[1] > 390:
        pyautogui.moveTo(1912, 1072)
    else:
        pyautogui.moveTo(1912, 8)
        


# To toggle status of control variables
def keyPressed(key):
    global perform
    global showCentroid
    global yellow_range,red_range,blue_range
    # toggle mouse simulation
    if key == ord('p'):
        perform = not perform
        if not perform:
            print('Mouse simulation OFF...')
        else:
            print('Mouse simulation ON...')

    # toggle display of centroids
    elif key == ord('c'):
        showCentroid = not showCentroid
        if not showCentroid:
            print('Dont Show Centroids...')
        else:
            print('Show Centroids...')

    elif key == ord('r'):
        print('	You have entered recalibration mode., Press Space when done')

        yellow_range = calibrateColor('Right hand Index Finger', yellow_range)
        red_range = calibrateColor('Left Hand Index Finger', red_range)
        blue_range = calibrateColor('Left hand Middle Finger', blue_range)

    else:
        pass

# cv2.inRange function is used to filter out a particular color from the frame
# The result then undergoes morphosis i.e. erosion and dilation 
def removeNoise(hsv_frame, color_Range):

    mask = cv2.inRange( hsv_frame, color_Range[0], color_Range[1])
    # Remove noise from the passed frame using morpholigical transformation
    eroded = cv2.erode( mask, kernel, iterations=1)
    dilated = cv2.dilate( eroded, kernel, iterations=1)

    return dilated

# Contour detection
def drawCentroid(vid, color_area, mask, showCentroid):

    contour, hierarchy = cv2.findContours( mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    l=len(contour)
    area = np.zeros(l)

    # filtering contours on the basis of area range which has been specified 
    for i in range(l):
        if cv2.contourArea(contour[i])>color_area[0] and cv2.contourArea(contour[i])<color_area[1]:
            area[i] = cv2.contourArea(contour[i])
        else:
            area[i] = 0

    a = sorted(area, reverse = True)

    # filtering out the max area contour 
    for i in range(l):
        for j in range(1):
            if area[i] == a[j]:
                contour[i], contour[j] = contour[j], contour[i]

    if l > 0 :
        return findCentroid(contour[0], showCentroid, vid)

    else:
        # handling error scenarios
        return (-1,-1)

# to be passed in createTrackBar function
def nothing(x):
    pass

# This function helps in filtering the required colored objects from the background
def calibrateColor(color, def_range):

    global kernel
    name = 'Calibrate '+ color
    cv2.namedWindow(name)
    cv2.createTrackbar('Hue', name, def_range[0][0] + 20, 180, nothing)
    cv2.createTrackbar('Sat', name, def_range[0][1]   , 255, nothing)
    cv2.createTrackbar('Val', name, def_range[0][2]   , 255, nothing)
    while(1):
        ret, frameinv = cap.read()
        frame = cv2.flip(frameinv ,1)
        # converting to hsv format 
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
        # getting the position of the track bar in each frame
        hue = cv2.getTrackbarPos('Hue', name)
        sat = cv2.getTrackbarPos('Sat', name)
        val = cv2.getTrackbarPos('Val', name)

        lower = np.array([hue-20,sat,val])
        upper = np.array([hue+20,255,255])
        
        # getting back the value after noise removal
        dilated = removeNoise(hsv, [lower, upper])

        cv2.imshow(name, dilated)

        k = cv2.waitKey(5) & 0xFF
        if k == ord(' '):
            cv2.destroyWindow(name)
            return np.array([[hue - 20,sat,val],[hue + 20,255,255]])
        elif k == ord('d'):
            cv2.destroyWindow(name)
            return def_range

# setting the cursor position by using current and temp position of the cursor
def setCursorPos( yc, pyp):

    yp = np.zeros(2)

    if abs(yc[0] - pyp[0]) < 5 and abs(yc[1] - pyp[1]) < 5:
        yp[0] = yc[0] + .7*(pyp[0] - yc[0]) 
        yp[1] = yc[1] + .7*(pyp[1] - yc[1])
    else:
        yp[0] = yc[0] + .1*(pyp[0] - yc[0])
        yp[1] = yc[1] + .1*(pyp[1] - yc[1])

    return yp

# Distance between two centroids
def distance( c1, c2):
    return pow((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2, 0.5)

# Depending upon the relative positions of the three centroids, this function chooses whether 
# the user desires free movement of cursor, left click, right click or dragging
def chooseAction(yp, rc, bc):
    out = np.array(['move', 'false'])
    if rc[0] != -1 and bc[0] != -1:

        if distance(yp,rc) < 50 and distance(yp, bc) < 50 and distance(rc, bc) < 50 :
            out[0] = 'drag'
            out[1] = 'true'
            return out
        elif distance(rc,bc) < 40: 
            out[0] = 'left'
            return out
        elif distance(yp,rc) < 40:
            out[0] = 'right'
            return out
        elif distance(yp,rc) > 40 and rc[1] - bc[1] > 120:
            out[0] = 'down'
            return out
        elif bc[1]-rc[1] > 110:
            out[0] = 'up'
            return out
        else:
            return out

    else:
        out[0] = -1
        return out 

# Movement of cursor on screen, left click, right click,scroll up, scroll down
# and dragging actions are performed here based on value stored in 'action'.  
def performAction( yp, rc, bc, action, drag, perform):
    if perform:
        cursor[0] = 4*(yp[0] - 110)
        cursor[1] = 4*(yp[1] - 120)
        if action == 'move':
            cursorHandler(yp, cursor)

        elif action == 'left':
            pyautogui.click(button = 'left')

        elif action == 'right':
            pyautogui.click(button = 'right')
            time.sleep(0.5)

        elif action == 'up':
            pyautogui.scroll(5)

        elif action == 'down':
            pyautogui.scroll(-5)

        elif action == 'drag' and drag == 'true':
            global y_pos
            drag = 'false'
            pyautogui.mouseDown()

            while(1):

                key = cv2.waitKey(10) & 0xFF
                keyPressed(key)

                flag, frameinv = cap.read()
                # get the mirror image of the frame for processing
                frame = cv2.flip( frameinv, 1)

                hsv = cv2.cvtColor( frame, cv2.COLOR_BGR2HSV)

                b_mask = removeNoise( hsv, blue_range)
                r_mask = removeNoise( hsv, red_range)
                y_mask = removeNoise( hsv, yellow_range)

                y_temp = y_pos 

                b_cen = drawCentroid( frame, b_area, b_mask, showCentroid)
                r_cen = drawCentroid( frame, r_area, r_mask, showCentroid)
                y_cen = drawCentroid( frame, y_area, y_mask, showCentroid)

                if y_temp[0]!=-1 and y_cen[0]!=-1:
                    y_pos = setCursorPos(y_cen, y_temp)

                performAction(y_pos,r_cen, b_cen, 'move', drag, perform)
                cv2.imshow('Gesture Control', frame)

                if distance(y_pos,r_cen)>60 or distance(y_pos,b_cen)>60 or distance(r_cen,b_cen)>60:
                    break

            pyautogui.mouseUp()
            

def mainHandler():
    
    global y_pos
    
    while(1):
        
        key = cv2.waitKey(10) & 0xFF
        keyPressed(key)

        flag, frameinv = cap.read()
        # get the mirror image of the frame for processing
        frame = cv2.flip( frameinv, 1)

        hsv = cv2.cvtColor( frame, cv2.COLOR_BGR2HSV)

        b_mask = removeNoise( hsv, blue_range)
        r_mask = removeNoise( hsv, red_range)
        y_mask = removeNoise( hsv, yellow_range)

        y_temp = y_pos 

        b_cen = drawCentroid( frame, b_area, b_mask, showCentroid)
        r_cen = drawCentroid( frame, r_area, r_mask, showCentroid)
        y_cen = drawCentroid( frame, y_area, y_mask, showCentroid)

        if y_temp[0]!=-1 and y_cen[0]!=-1 and y_pos[0]!=-1:
            y_pos = setCursorPos(y_cen, y_temp)

        output = chooseAction(y_pos, r_cen, b_cen)
        if output[0]!=-1:
            performAction(y_pos, r_cen, b_cen, output[0], output[1], perform)

        cv2.imshow('Gesture Control', frame)

        if key == 27:
            break





## Frame capture and start of gesture control

In [ ]:
cap = cv2.VideoCapture(0)

print('You have entered calibration mode, use the trackbars to calibrate and press SPACE when done.')

yellow_range = calibrateColor('Right hand Index Finger', yellow_range)
red_range = calibrateColor('Left Hand Index Finger', red_range)
blue_range = calibrateColor('Left hand Middle Finger', blue_range)
print('Calibration Successfull...')

# range array to pass to the main handler function
ranges = [blue_range, red_range, yellow_range]

cv2.namedWindow('Gesture Control')

print('Press P to turn ON and OFF mouse simulation, Press C to display the centroid of various colours.')
print('Press R to recalibrate color ranges.')
print('Press ESC to exit.')


if __name__=="__main__":
    mainHandler()


You have entered calibration mode, use the trackbars to calibrate and press SPACE when done.
Calibration Successfull...
Press P to turn ON and OFF mouse simulation, Press C to display the centroid of various colours.
Press R to recalibrate color ranges.
Press ESC to exit.


## Cleanup

In [ ]:
cv2.destroyAllWindows()
cap.release()